<a href="https://colab.research.google.com/github/peterbabulik/QuantumWalker/blob/main/Quantum_Kernel_SVM_Breast_Cancer_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit qiskit-ibm-runtime qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.9/359.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.5/69.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 8.0 MB/s eta 0:00:00


In [ ]:
import qiskit
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
import numpy as np
from typing import List, Optional, Tuple, Dict
import matplotlib.pyplot as plt
import time
import os

# Scikit-learn imports
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_breast_cancer # CHANGED DATASET
from sklearn.metrics import accuracy_score

# Qiskit Runtime imports
QISKIT_RUNTIME_VERSION = "Unknown"; qiskit_runtime_available = False
QiskitRuntimeService = Sampler = Session = generate_preset_pass_manager = IBMRuntimeBackend = None
try:
    from qiskit_ibm_runtime import QiskitRuntimeService,SamplerV2 as Sampler,Session,IBMBackend as IBMRuntimeBackend
    from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
    qiskit_runtime_available=True; import qiskit_ibm_runtime; QISKIT_RUNTIME_VERSION=qiskit_ibm_runtime.__version__
    print(f"Imported Qiskit Runtime {QISKIT_RUNTIME_VERSION}")
except ImportError: print("CRIT WARN: qiskit_ibm_runtime not imported.")
print(f"Qiskit (Terra) version: {qiskit.__version__}")

class QuantumKernelQPUAdapt:
    # ... (QuantumKernelQPUAdapt class definition - REMAINS IDENTICAL) ...
    def __init__(self, n_data_qubits: int):
        self.n_data_qubits = n_data_qubits
        self.aer_simulator = AerSimulator(method='statevector')
    def create_feature_map_circuit(self,x,name_suffix=""):
        if x.ndim!=1: raise ValueError(f"x must be 1D, got {x.shape}")
        qr=QuantumRegister(self.n_data_qubits,'q_d'); qc=QuantumCircuit(qr,name=f"fm_ry_{name_suffix}")
        for i in range(self.n_data_qubits):
            if i<len(x): qc.ry(x[i],qr[i]) # Only encode up to n_data_qubits features if x is longer
        return qc
    def get_statevector_aer(self,x):
        c=self.create_feature_map_circuit(x); c.save_statevector()
        tc=transpile(c,self.aer_simulator,optimization_level=0); r=self.aer_simulator.run(tc).result()
        return r.get_statevector(c).data
    def kernel_entry_aer(self,x1,x2):
        s1,s2=self.get_statevector_aer(x1),self.get_statevector_aer(x2)
        return np.abs(np.vdot(s1,s2))**2
    def _construct_fidelity_circuit(self,x1,x2):
        qc1=self.create_feature_map_circuit(x1,"x1");qc2=self.create_feature_map_circuit(x2,"x2")
        qc2d=qc2.inverse();qr_d=qc1.qregs[0];crn="c_fid"
        cr_d=ClassicalRegister(self.n_data_qubits,crn)
        fqc=QuantumCircuit(qr_d,cr_d,name=f"fid_{time.time_ns()}");fqc.append(qc1.to_instruction(),qr_d);fqc.append(qc2d.to_instruction(),qr_d)
        fqc.measure(qr_d,cr_d)
        return fqc,cr_d.name
    def _overlap_sq_qpu(self, x1, x2, sampler_instance_arg, qpu_backend_arg, shots_arg):
        fidelity_qc,cr_name_in_circuit=self._construct_fidelity_circuit(x1,x2)
        pm_opt_level=0
        if hasattr(qpu_backend_arg,'target')and qpu_backend_arg.target is not None:
            pm_fid=generate_preset_pass_manager(target=qpu_backend_arg.target,optimization_level=pm_opt_level)
        else:
            pm_fid=generate_preset_pass_manager(backend=qpu_backend_arg,optimization_level=pm_opt_level)
        transpiled_fidelity_qc=pm_fid.run(fidelity_qc)
        job=sampler_instance_arg.run([(transpiled_fidelity_qc,)],shots=shots_arg);result=job.result();overlap_sq_val=0.0
        if result and len(result)>0:
            pub_result=result[0];counts={}
            if hasattr(pub_result.data,cr_name_in_circuit):counts=getattr(pub_result.data,cr_name_in_circuit).get_counts()
            elif hasattr(pub_result.data,'meas'):counts=pub_result.data.meas.get_counts()
            if not counts:print(f"W:No counts({cr_name_in_circuit}/meas).Fields:{dir(pub_result.data)}")
            else:prob_0=counts.get('0'*self.n_data_qubits,0)/shots_arg;overlap_sq_val=prob_0
        else:print(f"E:QPU job kernel entry fail {x1}v{x2}")
        return overlap_sq_val
    def compute_fidelity_matrix(self,X_left_data,X_right_data=None,use_qpu=False,
                                sampler_instance=None,qpu_backend=None,shots_per_entry=1024):
        n_left=X_left_data.shape[0]; X_eval_right=X_right_data if X_right_data is not None else X_left_data
        n_right=X_eval_right.shape[0]; is_sym=(X_right_data is None)or(np.array_equal(X_left_data,X_eval_right)and X_left_data.shape==X_eval_right.shape)
        fidelity_matrix=np.zeros((n_left,n_right))
        if not use_qpu: # Aer Statevector Simulation
            for i in range(n_left):
                sj=i if is_sym else 0
                for j in range(sj,n_right):
                    if is_sym and i==j: val=1.0
                    else: val=self.kernel_entry_aer(X_left_data[i],X_eval_right[j])
                    fidelity_matrix[i,j]=val
                    if is_sym and i!=j: fidelity_matrix[j,i]=val
            return fidelity_matrix
        if sampler_instance is None or qpu_backend is None: raise ValueError("Sampler & QPU backend needed for QPU.")
        print(f"FidMat QPU ({self.n_data_qubits}Q data): Gen&Transpile circuits for {n_left}x{n_right} matrix...")
        circuits_to_run=[]; circuit_metadata_map=[]
        for i in range(n_left):
            sj=i if is_sym else 0
            for j in range(sj,n_right):
                if is_sym and i==j: fidelity_matrix[i,j]=1.0;
                else: fqc,crn=self._construct_fidelity_circuit(X_left_data[i],X_eval_right[j]); circuits_to_run.append(fqc); circuit_metadata_map.append({'ij':(i,j),'crn':crn})
        if not circuits_to_run:
            if is_sym: np.fill_diagonal(fidelity_matrix, 1.0)
            return fidelity_matrix
        pm_opt_fid=0
        if hasattr(qpu_backend,'target')and qpu_backend.target is not None: pm=generate_preset_pass_manager(target=qpu_backend.target,optimization_level=pm_opt_fid)
        else: pm=generate_preset_pass_manager(backend=qpu_backend,optimization_level=pm_opt_fid)
        print(f"Transpiling {len(circuits_to_run)} fid circuits..."); t_fqcs=pm.run(circuits_to_run)
        pubs=[(qc,)for qc in t_fqcs]; print(f"Submitting {len(pubs)} PUBs..."); job=sampler_instance.run(pubs,shots=shots_per_entry)
        job_id_fid=job.job_id(); print(f"Fid JobID:{job_id_fid} submitted. Wait..."); result=job.result(); print("Fid job done.")
        if result and len(result)==len(pubs):
            for k,pub_r in enumerate(result):
                mi,mj=circuit_metadata_map[k]['ij'];crne=circuit_metadata_map[k]['crn'];cts={};val=0.0
                if hasattr(pub_r.data,crne):cts=getattr(pub_r.data,crne).get_counts()
                elif hasattr(pub_r.data,'meas'):cts=pub_r.data.meas.get_counts()
                if not cts:print(f"W:No counts K({mi},{mj}).Data:{dir(pub_r.data)}")
                else:p0=cts.get('0'*self.n_data_qubits,0)/shots_per_entry;val=p0
                fidelity_matrix[mi,mj]=val
                if is_sym and mi!=mj:fidelity_matrix[mj,mi]=val
        else:print(f"E:QPU job fail/bad results.Exp{len(pubs)},got{len(result)if result else 0}.");fidelity_matrix.fill(np.nan)
        return fidelity_matrix
    def visualize_kernel_matrix(self,K,title_prefix="Kernel Matrix", kernel_type_for_title="linear"):
        plt.figure(figsize=(7,5));plt.imshow(K,cmap='viridis',vmin=0,vmax=1,aspect='auto')
        plt.colorbar(label="Kernel Value");plt.title(title_prefix+f"\n({self.n_data_qubits} data-Q, Type: {kernel_type_for_title})",fontsize=10)
        plt.xticks(fontsize=8);plt.yticks(fontsize=8);plt.tight_layout();plt.show()

def transform_fidelity_matrix(fidelity_matrix, kernel_type, gamma=1.0, degree=2, offset=0.0):
    if kernel_type=='linear':return fidelity_matrix
    elif kernel_type=='rbf':return np.exp(-gamma*(1.0-fidelity_matrix))
    elif kernel_type=='polynomial':return(fidelity_matrix+offset)**degree
    else:raise ValueError(f"Unknown kernel_type: {kernel_type}")

# --- IBM Quantum Setup (NEW TOKEN) ---
IBM_QUANTUM_TOKEN_DIRECT = '8a1372e040a117b3151a15d9eddcbf6e05b6c6b3c69bb1a60a8c7da5a956d7a2c8696a3a5ccc7111818804b0880ecaeefb15882e9633a34cd45259a16ced8a35'
IBM_QUANTUM_CHANNEL_DIRECT = 'ibm_quantum'; IBM_QUANTUM_INSTANCE_DIRECT = 'ibm-q/open/main'; QPU_BACKEND_NAME = 'ibm_brisbane'
service=None; qpu_backend_for_kernel=None
if qiskit_runtime_available:
    try: service=QiskitRuntimeService(channel=IBM_QUANTUM_CHANNEL_DIRECT,instance=IBM_QUANTUM_INSTANCE_DIRECT,token=IBM_QUANTUM_TOKEN_DIRECT); print("Service init.")
    except Exception as e:print(f"ERR Service:{e}")
    if service:
        try: qpu_bk_obj=service.backend(QPU_BACKEND_NAME); qpu_backend_for_kernel=qpu_bk_obj if qpu_bk_obj else None; print(f"Selected QPU:{qpu_backend_for_kernel.name}({qpu_backend_for_kernel.status().status_msg})"if qpu_backend_for_kernel else"QPU not found")
        except Exception as e:print(f"ERR QPU Backend:{e}")
if qpu_backend_for_kernel is None and qiskit_runtime_available : print(f"Warning: QPU {QPU_BACKEND_NAME} not loaded.")

# --- Main Execution Block ---
if __name__ == "__main__":
    print("\n=== Quantum Kernel SVM (Breast Cancer Dataset) ===")
    # --- Control Flags ---
    RUN_QPU_PART = True
    PLOT_KERNEL_MATRICES = False # Set to True to visualize all kernel matrices
    N_FEATURES_TO_USE = 4 # Number of features to select from dataset for the kernel
                          # This will be n_data_qubits for our kernel

    # 1. Load and prepare dataset
    cancer_data = load_breast_cancer()
    X_raw, y_data = cancer_data.data, cancer_data.target

    # Select a subset of features (e.g., the first N_FEATURES_TO_USE)
    X_selected_features = X_raw[:, :N_FEATURES_TO_USE]

    # Scale features to [0, pi] for Ry encoding
    scaler = MinMaxScaler(feature_range=(0, np.pi))
    X_data_scaled = scaler.fit_transform(X_selected_features)

    # Take a subset of samples for manageable execution time
    N_TOTAL_SAMPLES_TO_USE = 50 # Total samples from the dataset to use
    if X_data_scaled.shape[0] > N_TOTAL_SAMPLES_TO_USE:
        # Stratified sampling to maintain class proportions might be better here
        # For simplicity, just taking the first N samples and shuffling before split
        indices = np.arange(X_data_scaled.shape[0])
        np.random.shuffle(indices) # Shuffle to get a random subset
        X_subset = X_data_scaled[indices[:N_TOTAL_SAMPLES_TO_USE]]
        y_subset = y_data[indices[:N_TOTAL_SAMPLES_TO_USE]]
    else:
        X_subset = X_data_scaled
        y_subset = y_data

    X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(
        X_subset, y_subset, test_size=0.4, random_state=123, stratify=y_subset
    )
    print(f"Full Dataset Used (after selection & subsetting): {X_train_full.shape[0]} train, {X_test_full.shape[0]} test ({X_train_full.shape[1]} features).")

    # Define QPU subset sizes (from the X_train_full, X_test_full created above)
    QPU_TRAIN_SAMPLES = 10
    QPU_TEST_SAMPLES = 5
    X_train_qpu_subset = X_train_full[:min(QPU_TRAIN_SAMPLES, X_train_full.shape[0])]
    y_train_qpu_subset = y_train_full[:min(QPU_TRAIN_SAMPLES, X_train_full.shape[0])]
    X_test_qpu_subset = X_test_full[:min(QPU_TEST_SAMPLES, X_test_full.shape[0])]
    y_test_qpu_subset = y_test_full[:min(QPU_TEST_SAMPLES, X_test_full.shape[0])]
    print(f"QPU Subsets to be Used: {X_train_qpu_subset.shape[0]} train, {X_test_qpu_subset.shape[0]} test.")

    qkernel_instance = QuantumKernelQPUAdapt(n_data_qubits=N_FEATURES_TO_USE) # n_data_qubits = features used
    print(f"\nKernel Instance: {qkernel_instance.n_data_qubits}Q data (Fidelity based).")

    print("\n--- Classical SVM with RBF Kernel (Scikit-learn) ---")
    svm_classical_rbf = SVC(kernel='rbf', C=1.0, gamma='scale'); svm_classical_rbf.fit(X_train_full, y_train_full)
    y_pred_classical_rbf = svm_classical_rbf.predict(X_test_full); accuracy_classical_rbf = accuracy_score(y_test_full, y_pred_classical_rbf)
    print(f"Classical SVM (RBF Kernel) Accuracy on full test set: {accuracy_classical_rbf:.4f}")
    results_log = [{'kernel': 'Classical RBF', 'source': 'Sklearn', 'accuracy': accuracy_classical_rbf, 'test_set_size': X_test_full.shape[0]}]

    print("\n--- Computing Fidelity Matrices on AerSimulator (using full dataset) ---")
    F_train_aer = qkernel_instance.compute_fidelity_matrix(X_train_full)
    F_test_aer  = qkernel_instance.compute_fidelity_matrix(X_test_full, X_train_full)
    print("F_train_aer shape:",F_train_aer.shape,"; F_test_aer shape:",F_test_aer.shape)

    kernel_types_to_test = ['linear', 'rbf', 'polynomial']
    for k_type in kernel_types_to_test:
        print(f"\n--- Testing Quantum Kernel Type: {k_type} (Aer) ---")
        K_train_aer_transformed = transform_fidelity_matrix(F_train_aer, k_type)
        K_test_aer_transformed  = transform_fidelity_matrix(F_test_aer, k_type)
        if PLOT_KERNEL_MATRICES: qkernel_instance.visualize_kernel_matrix(K_train_aer_transformed, f"K(FullTr,FullTr)-Aer", k_type)

        svm_aer = SVC(kernel='precomputed', C=1.0); svm_aer.fit(K_train_aer_transformed, y_train_full)
        y_pred_aer = svm_aer.predict(K_test_aer_transformed); accuracy_aer = accuracy_score(y_test_full, y_pred_aer)
        print(f"Quantum SVM (Aer, {k_type}) Accuracy on full test set: {accuracy_aer:.4f}")
        results_log.append({'kernel': f"Quantum {k_type}", 'source': 'Aer', 'accuracy': accuracy_aer, 'test_set_size': X_test_full.shape[0]})

    F_train_qpu, F_test_qpu = None, None
    qpu_ready = qpu_backend_for_kernel and hasattr(qpu_backend_for_kernel,'status') and qpu_backend_for_kernel.status().operational

    if RUN_QPU_PART and qpu_ready and qiskit_runtime_available:
        print(f"\n--- Computing Fidelity Matrices on QPU: {qpu_backend_for_kernel.name} (using QPU subset) ---")
        shots_qpu = 1024
        samp_opts = {"dynamical_decoupling":{"enable":True,"sequence_type":"XX"}, "twirling":{"enable_gates":True,"enable_measure":True}}
        print(f"Sampler opts:{samp_opts}")

        with Session(backend=qpu_backend_for_kernel) as session:
            sampler_qpu = Sampler(options=samp_opts)
            print(f"\nF_train_qpu = Fidelity(X_train_qpu_subset({X_train_qpu_subset.shape[0]}), X_train_qpu_subset) on QPU...")
            F_train_qpu = qkernel_instance.compute_fidelity_matrix(X_train_qpu_subset, use_qpu=True,
                sampler_instance=sampler_qpu, qpu_backend=qpu_backend_for_kernel, shots_per_entry=shots_qpu)
            if F_train_qpu is not None and not np.isnan(F_train_qpu).all(): print("F_train_qpu computed. Shape:", F_train_qpu.shape)
            else: print("F_train_qpu computation failed/returned NaNs.")

            if X_test_qpu_subset.shape[0] > 0 and F_train_qpu is not None and not np.isnan(F_train_qpu).all():
                print(f"\nF_test_qpu = Fidelity(X_test_qpu_subset({X_test_qpu_subset.shape[0]}), X_train_qpu_subset) on QPU...")
                F_test_qpu = qkernel_instance.compute_fidelity_matrix(X_test_qpu_subset, X_train_qpu_subset, use_qpu=True,
                    sampler_instance=sampler_qpu, qpu_backend=qpu_backend_for_kernel, shots_per_entry=shots_qpu)
                if F_test_qpu is not None and not np.isnan(F_test_qpu).all(): print("F_test_qpu computed. Shape:", F_test_qpu.shape)
                else: print("F_test_qpu computation failed/returned NaNs.")
        print("--- QPU Fidelity Matrix Computation Complete ---")

        if F_train_qpu is not None and F_test_qpu is not None and not np.isnan(F_train_qpu).any() and not np.isnan(F_test_qpu).any():
            for k_type in kernel_types_to_test:
                print(f"\n--- Testing Quantum Kernel Type: {k_type} (QPU) ---")
                K_train_qpu_transformed = transform_fidelity_matrix(F_train_qpu, k_type)
                K_test_qpu_transformed  = transform_fidelity_matrix(F_test_qpu, k_type)
                if PLOT_KERNEL_MATRICES: qkernel_instance.visualize_kernel_matrix(K_train_qpu_transformed, f"K(QPU_Tr,QPU_Tr)-QPU", k_type)

                svm_qpu = SVC(kernel='precomputed',C=1.0); svm_qpu.fit(K_train_qpu_transformed,y_train_qpu_subset)
                y_pred_qpu = svm_qpu.predict(K_test_qpu_transformed); accuracy_qpu = accuracy_score(y_test_qpu_subset,y_pred_qpu)
                print(f"Quantum SVM (QPU, {k_type}) Accuracy on QPU test subset: {accuracy_qpu:.4f}")
                results_log.append({'kernel': f"Quantum {k_type}", 'source': 'QPU', 'accuracy': accuracy_qpu, 'test_set_size': X_test_qpu_subset.shape[0]})
        else: print("QPU fidelity matrices not suitable for SVM.")
    elif RUN_QPU_PART: print(f"\nQPU {QPU_BACKEND_NAME} not ready or runtime not available. Skipping QPU part.")
    else: print ("\nRUN_QPU_PART flag is False. Skipping QPU part.")

    print("\n\n--- SVM Accuracy Summary ---")
    print("-----------------------------------------------------------------")
    print(f"{'Kernel Type':<20} | {'Source':<10} | {'Accuracy':<10} | {'Test N':<7}")
    print("-----------------------------------------------------------------")
    for res_item in results_log:
        print(f"{res_item['kernel']:<20} | {res_item['source']:<10} | {res_item['accuracy']:.4f}   | {res_item.get('test_set_size', 'N/A')}")
    print("-----------------------------------------------------------------")

    # Decision boundary plotting is skipped by PLOT_DECISION_BOUNDARIES = False
    if PLOT_DECISION_BOUNDARIES and N_FEATURES_TO_USE == 2:
         print("\nDecision boundary plotting is enabled but can be slow...")
         # ... (plotting logic can be inserted here if desired for a specific kernel)
    else:
        print("\nDecision boundary plotting skipped (PLOT_DECISION_BOUNDARIES=False or n_features != 2).")

    print("\n=== Demo Complete ===")
    if RUN_QPU_PART and qpu_ready: print(f"REMEMBER token {IBM_QUANTUM_TOKEN_DIRECT[:10]}...")

Imported Qiskit Runtime 0.39.0
Qiskit (Terra) version: 2.0.1


<ipython-input-2-4e87be5d892c>:128: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, "ibm_cloud" and "local" will be the only valid channels. For information on migrating to the new IBM Quantum Platform on the "ibm_cloud" channel, review the migration guide https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp .
  try: service=QiskitRuntimeService(channel=IBM_QUANTUM_CHANNEL_DIRECT,instance=IBM_QUANTUM_INSTANCE_DIRECT,token=IBM_QUANTUM_TOKEN_DIRECT); print("Service init.")


Service init.
Selected QPU:ibm_brisbane(active)

=== Quantum Kernel SVM (Breast Cancer Dataset) ===
Full Dataset Used (after selection & subsetting): 30 train, 20 test (4 features).
QPU Subsets to be Used: 10 train, 5 test.

Kernel Instance: 4Q data (Fidelity based).

--- Classical SVM with RBF Kernel (Scikit-learn) ---
Classical SVM (RBF Kernel) Accuracy on full test set: 0.8000

--- Computing Fidelity Matrices on AerSimulator (using full dataset) ---
F_train_aer shape: (30, 30) ; F_test_aer shape: (20, 30)

--- Testing Quantum Kernel Type: linear (Aer) ---
Quantum SVM (Aer, linear) Accuracy on full test set: 0.8500

--- Testing Quantum Kernel Type: rbf (Aer) ---
Quantum SVM (Aer, rbf) Accuracy on full test set: 0.9500

--- Testing Quantum Kernel Type: polynomial (Aer) ---
Quantum SVM (Aer, polynomial) Accuracy on full test set: 0.8500

--- Computing Fidelity Matrices on QPU: ibm_brisbane (using QPU subset) ---
Sampler opts:{'dynamical_decoupling': {'enable': True, 'sequence_type': 'X

NameError: name 'PLOT_DECISION_BOUNDARIES' is not defined

In [ ]:
import qiskit
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator # ENSURED IMPORT
import numpy as np
from typing import List, Optional, Tuple, Dict
import matplotlib.pyplot as plt
import time
import os

# Scikit-learn imports
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_breast_cancer # CHANGED DATASET
from sklearn.metrics import accuracy_score

# Qiskit Runtime imports
QISKIT_RUNTIME_VERSION = "Unknown"; qiskit_runtime_available = False
QiskitRuntimeService = Sampler = Session = generate_preset_pass_manager = IBMRuntimeBackend = None
try:
    from qiskit_ibm_runtime import QiskitRuntimeService,SamplerV2 as Sampler,Session,IBMBackend as IBMRuntimeBackend
    from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
    qiskit_runtime_available=True; import qiskit_ibm_runtime; QISKIT_RUNTIME_VERSION=qiskit_ibm_runtime.__version__
    print(f"Imported Qiskit Runtime {QISKIT_RUNTIME_VERSION}")
except ImportError: print("CRIT WARN: qiskit_ibm_runtime not imported.")
print(f"Qiskit (Terra) version: {qiskit.__version__}")

class QuantumKernelQPUAdapt:
    # ... (Class definition exactly as in the previous successful run) ...
    def __init__(self, n_data_qubits: int):
        self.n_data_qubits = n_data_qubits
        self.aer_simulator = AerSimulator(method='statevector')
    def create_feature_map_circuit(self,x,name_suffix=""):
        if x.ndim!=1: raise ValueError(f"x must be 1D, got {x.shape}")
        qr=QuantumRegister(self.n_data_qubits,'q_d'); qc=QuantumCircuit(qr,name=f"fm_ry_{name_suffix}")
        for i in range(self.n_data_qubits):
            if i<len(x): qc.ry(x[i],qr[i])
        return qc
    def get_statevector_aer(self,x):
        c=self.create_feature_map_circuit(x); c.save_statevector()
        tc=transpile(c,self.aer_simulator,optimization_level=0); r=self.aer_simulator.run(tc).result()
        return r.get_statevector(c).data
    def kernel_entry_aer(self,x1,x2):
        s1,s2=self.get_statevector_aer(x1),self.get_statevector_aer(x2)
        return np.abs(np.vdot(s1,s2))**2
    def _construct_fidelity_circuit(self,x1,x2):
        qc1=self.create_feature_map_circuit(x1,"x1");qc2=self.create_feature_map_circuit(x2,"x2")
        qc2d=qc2.inverse();qr_d=qc1.qregs[0];crn="c_fid"
        cr_d=ClassicalRegister(self.n_data_qubits,crn)
        fqc=QuantumCircuit(qr_d,cr_d,name=f"fid_{time.time_ns()}");fqc.append(qc1.to_instruction(),qr_d);fqc.append(qc2d.to_instruction(),qr_d)
        fqc.measure(qr_d,cr_d)
        return fqc,cr_d.name
    def _overlap_sq_qpu(self, x1, x2, sampler_instance_arg, qpu_backend_arg, shots_arg):
        fidelity_qc,cr_name_in_circuit=self._construct_fidelity_circuit(x1,x2)
        pm_opt_level=0
        if hasattr(qpu_backend_arg,'target')and qpu_backend_arg.target is not None:
            pm_fid=generate_preset_pass_manager(target=qpu_backend_arg.target,optimization_level=pm_opt_level)
        else:
            pm_fid=generate_preset_pass_manager(backend=qpu_backend_arg,optimization_level=pm_opt_level)
        transpiled_fidelity_qc=pm_fid.run(fidelity_qc)
        job=sampler_instance_arg.run([(transpiled_fidelity_qc,)],shots=shots_arg);result=job.result();overlap_sq_val=0.0
        if result and len(result)>0:
            pub_result=result[0];counts={}
            if hasattr(pub_result.data,cr_name_in_circuit):counts=getattr(pub_result.data,cr_name_in_circuit).get_counts()
            elif hasattr(pub_result.data,'meas'):counts=pub_result.data.meas.get_counts()
            if not counts:print(f"W:No counts({cr_name_in_circuit}/meas).Fields:{dir(pub_result.data)}")
            else:prob_0=counts.get('0'*self.n_data_qubits,0)/shots_arg;overlap_sq_val=prob_0
        else:print(f"E:QPU job kernel entry fail {x1}v{x2}")
        return overlap_sq_val
    def compute_fidelity_matrix(self,X_left_data,X_right_data=None,use_qpu=False,
                                sampler_instance=None,qpu_backend=None,shots_per_entry=1024):
        n_left=X_left_data.shape[0]; X_eval_right=X_right_data if X_right_data is not None else X_left_data
        n_right=X_eval_right.shape[0]; is_sym=(X_right_data is None)or(np.array_equal(X_left_data,X_eval_right)and X_left_data.shape==X_eval_right.shape)
        fidelity_matrix=np.zeros((n_left,n_right))
        if not use_qpu:
            for i in range(n_left):
                sj=i if is_sym else 0
                for j in range(sj,n_right):
                    if is_sym and i==j: val=1.0
                    else: val=self.kernel_entry_aer(X_left_data[i],X_eval_right[j])
                    fidelity_matrix[i,j]=val
                    if is_sym and i!=j: fidelity_matrix[j,i]=val
            return fidelity_matrix
        if sampler_instance is None or qpu_backend is None: raise ValueError("Sampler & QPU backend needed for QPU.")
        print(f"FidMat QPU ({self.n_data_qubits}Q data): Gen&Transpile circuits for {n_left}x{n_right} matrix...")
        circuits_to_run=[]; circuit_metadata_map=[]
        for i in range(n_left):
            sj=i if is_sym else 0
            for j in range(sj,n_right):
                if is_sym and i==j: fidelity_matrix[i,j]=1.0;
                else: fqc,crn=self._construct_fidelity_circuit(X_left_data[i],X_eval_right[j]); circuits_to_run.append(fqc); circuit_metadata_map.append({'ij':(i,j),'crn':crn})
        if not circuits_to_run:
            if is_sym: np.fill_diagonal(fidelity_matrix, 1.0)
            return fidelity_matrix
        pm_opt_fid=0
        if hasattr(qpu_backend,'target')and qpu_backend.target is not None: pm=generate_preset_pass_manager(target=qpu_backend.target,optimization_level=pm_opt_fid)
        else: pm=generate_preset_pass_manager(backend=qpu_backend,optimization_level=pm_opt_fid)
        print(f"Transpiling {len(circuits_to_run)} fid circuits..."); t_fqcs=pm.run(circuits_to_run)
        pubs=[(qc,)for qc in t_fqcs]; print(f"Submitting {len(pubs)} PUBs..."); job=sampler_instance.run(pubs,shots=shots_per_entry)
        job_id_fid=job.job_id(); print(f"Fid JobID:{job_id_fid} submitted. Wait..."); result=job.result(); print("Fid job done.")
        if result and len(result)==len(pubs):
            for k,pub_r in enumerate(result):
                mi,mj=circuit_metadata_map[k]['ij'];crne=circuit_metadata_map[k]['crn'];cts={};val=0.0
                if hasattr(pub_r.data,crne):cts=getattr(pub_r.data,crne).get_counts()
                elif hasattr(pub_r.data,'meas'):cts=pub_r.data.meas.get_counts()
                if not cts:print(f"W:No counts K({mi},{mj}).Data:{dir(pub_r.data)}")
                else:p0=cts.get('0'*self.n_data_qubits,0)/shots_per_entry;val=p0
                fidelity_matrix[mi,mj]=val
                if is_sym and mi!=mj:fidelity_matrix[mj,mi]=val
        else:print(f"E:QPU job fail/bad results.Exp{len(pubs)},got{len(result)if result else 0}.");fidelity_matrix.fill(np.nan)
        return fidelity_matrix
    def visualize_kernel_matrix(self,K,title_prefix="Kernel Matrix", kernel_type_for_title="linear"):
        plt.figure(figsize=(7,5));plt.imshow(K,cmap='viridis',vmin=0,vmax=1,aspect='auto')
        plt.colorbar(label="Kernel Value");plt.title(title_prefix+f"\n({self.n_data_qubits} data-Q, Type: {kernel_type_for_title})",fontsize=10)
        plt.xticks(fontsize=8);plt.yticks(fontsize=8);plt.tight_layout();plt.show()

def transform_fidelity_matrix(fidelity_matrix, kernel_type, gamma=1.0, degree=2, offset=0.0):
    if kernel_type=='linear':return fidelity_matrix
    elif kernel_type=='rbf':return np.exp(-gamma*(1.0-fidelity_matrix))
    elif kernel_type=='polynomial':return(fidelity_matrix+offset)**degree
    else:raise ValueError(f"Unknown kernel_type: {kernel_type}")

# --- IBM Quantum Setup ---
IBM_QUANTUM_TOKEN_DIRECT = '8a1372e040a117b3151a15d9eddcbf6e05b6c6b3c69bb1a60a8c7da5a956d7a2c8696a3a5ccc7111818804b0880ecaeefb15882e9633a34cd45259a16ced8a35'
IBM_QUANTUM_CHANNEL_DIRECT = 'ibm_quantum'; IBM_QUANTUM_INSTANCE_DIRECT = 'ibm-q/open/main'; QPU_BACKEND_NAME = 'ibm_brisbane'
service=None; qpu_backend_for_kernel=None
if qiskit_runtime_available:
    try: service=QiskitRuntimeService(channel=IBM_QUANTUM_CHANNEL_DIRECT,instance=IBM_QUANTUM_INSTANCE_DIRECT,token=IBM_QUANTUM_TOKEN_DIRECT); print("Service init.")
    except Exception as e:print(f"ERR Service:{e}")
    if service:
        try: qpu_bk_obj=service.backend(QPU_BACKEND_NAME); qpu_backend_for_kernel=qpu_bk_obj if qpu_bk_obj else None; print(f"Selected QPU:{qpu_backend_for_kernel.name}({qpu_backend_for_kernel.status().status_msg})"if qpu_backend_for_kernel else"QPU not found")
        except Exception as e:print(f"ERR QPU Backend:{e}")
if qpu_backend_for_kernel is None and qiskit_runtime_available : print(f"Warning: QPU {QPU_BACKEND_NAME} not loaded.")

# --- Main Execution Block ---
if __name__ == "__main__":
    print("\n=== Quantum Kernel SVM (Breast Cancer Dataset) ===")
    # --- Control Flags ---
    RUN_QPU_PART = True
    PLOT_KERNEL_MATRICES = False
    PLOT_DECISION_BOUNDARIES = False # ENSURE THIS IS DEFINED

    N_FEATURES_TO_USE = 4
    N_TOTAL_SAMPLES_TO_USE = 50
    QPU_TRAIN_SAMPLES = 10
    QPU_TEST_SAMPLES = 5

    cancer_data = load_breast_cancer(); X_raw,y_data = cancer_data.data,cancer_data.target
    X_selected_features = X_raw[:, :N_FEATURES_TO_USE]
    scaler = MinMaxScaler(feature_range=(0, np.pi)); X_data_scaled = scaler.fit_transform(X_selected_features)
    if X_data_scaled.shape[0] > N_TOTAL_SAMPLES_TO_USE:
        indices = np.arange(X_data_scaled.shape[0]); np.random.shuffle(indices)
        X_subset,y_subset = X_data_scaled[indices[:N_TOTAL_SAMPLES_TO_USE]],y_data[indices[:N_TOTAL_SAMPLES_TO_USE]]
    else: X_subset,y_subset = X_data_scaled,y_data
    X_train_full,X_test_full,y_train_full,y_test_full = train_test_split(X_subset,y_subset,test_size=0.4,random_state=123,stratify=y_subset)
    print(f"Full Dataset Used: {X_train_full.shape[0]} train, {X_test_full.shape[0]} test ({X_train_full.shape[1]} features).")
    X_train_qpu_subset=X_train_full[:min(QPU_TRAIN_SAMPLES,X_train_full.shape[0])]; y_train_qpu_subset=y_train_full[:min(QPU_TRAIN_SAMPLES,X_train_full.shape[0])]
    X_test_qpu_subset=X_test_full[:min(QPU_TEST_SAMPLES,X_test_full.shape[0])]; y_test_qpu_subset=y_test_full[:min(QPU_TEST_SAMPLES,X_test_full.shape[0])]
    print(f"QPU Subsets Used: {X_train_qpu_subset.shape[0]} train, {X_test_qpu_subset.shape[0]} test.")

    # (Dataset plotting code can be here if PLOT_DECISION_BOUNDARIES is True or for small datasets)

    qkernel_instance = QuantumKernelQPUAdapt(N_FEATURES_TO_USE)
    print(f"\nKernel Instance: {qkernel_instance.n_data_qubits}Q data (Fidelity based).")

    print("\n--- Classical SVM with RBF Kernel (Scikit-learn) ---")
    svm_classical_rbf = SVC(kernel='rbf',C=1.0,gamma='scale'); svm_classical_rbf.fit(X_train_full,y_train_full)
    y_pred_classical_rbf = svm_classical_rbf.predict(X_test_full); accuracy_classical_rbf = accuracy_score(y_test_full,y_pred_classical_rbf)
    print(f"Classical SVM (RBF) Accuracy on full test set: {accuracy_classical_rbf:.4f}")
    results_log = [{'kernel':'Classical RBF','source':'Sklearn','accuracy':accuracy_classical_rbf,'test_set_size':X_test_full.shape[0]}]

    print("\n--- Computing Fidelity Matrices on AerSimulator (using full dataset) ---")
    F_train_aer = qkernel_instance.compute_fidelity_matrix(X_train_full)
    F_test_aer  = qkernel_instance.compute_fidelity_matrix(X_test_full, X_train_full)
    print("F_train_aer shape:",F_train_aer.shape,"; F_test_aer shape:",F_test_aer.shape)

    kernel_types_to_test = ['linear', 'rbf', 'polynomial']
    for k_type in kernel_types_to_test:
        print(f"\n--- Testing Quantum Kernel Type: {k_type} (Aer) ---")
        K_train_aer_transformed = transform_fidelity_matrix(F_train_aer, k_type)
        K_test_aer_transformed  = transform_fidelity_matrix(F_test_aer, k_type)
        if PLOT_KERNEL_MATRICES: qkernel_instance.visualize_kernel_matrix(K_train_aer_transformed, f"K(FullTr,FullTr)-Aer", k_type)
        svm_aer = SVC(kernel='precomputed',C=1.0); svm_aer.fit(K_train_aer_transformed,y_train_full)
        y_pred_aer = svm_aer.predict(K_test_aer_transformed); accuracy_aer = accuracy_score(y_test_full,y_pred_aer)
        print(f"Quantum SVM (Aer, {k_type}) Accuracy on full test set: {accuracy_aer:.4f}")
        results_log.append({'kernel':f"Quantum {k_type}",'source':'Aer','accuracy':accuracy_aer,'test_set_size':X_test_full.shape[0]})

    F_train_qpu, F_test_qpu = None, None
    qpu_ready = qpu_backend_for_kernel and hasattr(qpu_backend_for_kernel,'status') and qpu_backend_for_kernel.status().operational

    if RUN_QPU_PART and qpu_ready and qiskit_runtime_available:
        print(f"\n--- Computing Fidelity Matrices on QPU: {qpu_backend_for_kernel.name} (using QPU subset) ---")
        shots_qpu = 4096 # INCREASED SHOTS FOR QPU
        samp_opts = {"dynamical_decoupling":{"enable":True,"sequence_type":"XX"}, "twirling":{"enable_gates":True,"enable_measure":True}}
        print(f"Sampler opts:{samp_opts}")

        with Session(backend=qpu_backend_for_kernel) as session:
            sampler_qpu = Sampler(options=samp_opts)
            print(f"\nF_train_qpu = Fidelity(X_train_qpu_subset({X_train_qpu_subset.shape[0]}), X_train_qpu_subset) on QPU...")
            F_train_qpu = qkernel_instance.compute_fidelity_matrix(X_train_qpu_subset, use_qpu=True,
                sampler_instance=sampler_qpu, qpu_backend=qpu_backend_for_kernel, shots_per_entry=shots_qpu)
            if F_train_qpu is not None and not np.isnan(F_train_qpu).all(): print("F_train_qpu computed. Shape:", F_train_qpu.shape)
            else: print("F_train_qpu failed/NaNs.")

            if X_test_qpu_subset.shape[0]>0 and F_train_qpu is not None and not np.isnan(F_train_qpu).all():
                print(f"\nF_test_qpu = Fidelity(X_test_qpu_subset({X_test_qpu_subset.shape[0]}), X_train_qpu_subset) on QPU...")
                F_test_qpu = qkernel_instance.compute_fidelity_matrix(X_test_qpu_subset, X_train_qpu_subset, use_qpu=True,
                    sampler_instance=sampler_qpu, qpu_backend=qpu_backend_for_kernel, shots_per_entry=shots_qpu)
                if F_test_qpu is not None and not np.isnan(F_test_qpu).all(): print("F_test_qpu computed. Shape:", F_test_qpu.shape)
                else: print("F_test_qpu failed/NaNs.")
        print("--- QPU Fidelity Matrix Computation Complete ---")

        if F_train_qpu is not None and F_test_qpu is not None and not np.isnan(F_train_qpu).any() and not np.isnan(F_test_qpu).any():
            for k_type in kernel_types_to_test:
                print(f"\n--- Testing Quantum Kernel Type: {k_type} (QPU) ---")
                K_train_qpu_transformed = transform_fidelity_matrix(F_train_qpu, k_type)
                K_test_qpu_transformed  = transform_fidelity_matrix(F_test_qpu, k_type)
                if PLOT_KERNEL_MATRICES: qkernel_instance.visualize_kernel_matrix(K_train_qpu_transformed, f"K(QPU_Tr,QPU_Tr)-QPU", k_type)
                svm_qpu = SVC(kernel='precomputed',C=1.0); svm_qpu.fit(K_train_qpu_transformed,y_train_qpu_subset)
                y_pred_qpu = svm_qpu.predict(K_test_qpu_transformed); accuracy_qpu = accuracy_score(y_test_qpu_subset,y_pred_qpu)
                print(f"Quantum SVM (QPU, {k_type}) Accuracy on QPU test subset: {accuracy_qpu:.4f}")
                results_log.append({'kernel':f"Quantum {k_type}",'source':'QPU','accuracy':accuracy_qpu,'test_set_size':X_test_qpu_subset.shape[0]})
        else: print("QPU fidelity matrices not suitable for SVM.")
    elif RUN_QPU_PART: print(f"\nQPU {QPU_BACKEND_NAME} not ready or runtime not available. Skipping.")
    else: print ("\nRUN_QPU_PART flag is False. Skipping QPU part.")

    print("\n\n--- SVM Accuracy Summary ---") # ... (Summary print unchanged) ...
    print("-----------------------------------------------------------------");print(f"{'Kernel Type':<20} | {'Source':<10} | {'Accuracy':<10} | {'Test N':<7}");print("-----------------------------------------------------------------")
    for res_item in results_log: print(f"{res_item['kernel']:<20} | {res_item['source']:<10} | {res_item['accuracy']:.4f}   | {res_item.get('test_set_size','N/A')}")
    print("-----------------------------------------------------------------")

    if PLOT_DECISION_BOUNDARIES and N_FEATURES_TO_USE == 2: # ... (Decision boundary plotting - unchanged) ...
        print("\n--- Plotting Decision Boundaries ---") # ...
    else: print("\nDecision boundary plotting skipped.")

    print("\n=== Demo Complete ===")
    if RUN_QPU_PART and qpu_ready: print(f"REMEMBER token {IBM_QUANTUM_TOKEN_DIRECT[:10]}...")

Imported Qiskit Runtime 0.39.0
Qiskit (Terra) version: 2.0.1


<ipython-input-3-65104c561051>:128: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, "ibm_cloud" and "local" will be the only valid channels. For information on migrating to the new IBM Quantum Platform on the "ibm_cloud" channel, review the migration guide https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp .
  try: service=QiskitRuntimeService(channel=IBM_QUANTUM_CHANNEL_DIRECT,instance=IBM_QUANTUM_INSTANCE_DIRECT,token=IBM_QUANTUM_TOKEN_DIRECT); print("Service init.")


Service init.
Selected QPU:ibm_brisbane(active)

=== Quantum Kernel SVM (Breast Cancer Dataset) ===
Full Dataset Used: 30 train, 20 test (4 features).
QPU Subsets Used: 10 train, 5 test.

Kernel Instance: 4Q data (Fidelity based).

--- Classical SVM with RBF Kernel (Scikit-learn) ---
Classical SVM (RBF) Accuracy on full test set: 0.9000

--- Computing Fidelity Matrices on AerSimulator (using full dataset) ---
F_train_aer shape: (30, 30) ; F_test_aer shape: (20, 30)

--- Testing Quantum Kernel Type: linear (Aer) ---
Quantum SVM (Aer, linear) Accuracy on full test set: 0.9000

--- Testing Quantum Kernel Type: rbf (Aer) ---
Quantum SVM (Aer, rbf) Accuracy on full test set: 0.9000

--- Testing Quantum Kernel Type: polynomial (Aer) ---
Quantum SVM (Aer, polynomial) Accuracy on full test set: 0.9000

--- Computing Fidelity Matrices on QPU: ibm_brisbane (using QPU subset) ---
Sampler opts:{'dynamical_decoupling': {'enable': True, 'sequence_type': 'XX'}, 'twirling': {'enable_gates': True, 'ena